In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
data=pd.read_csv('/kaggle/input/data-in-brief/comment_data.csv')


In [3]:
import pandas as pd

# Load the CSV file
csv_file = '/kaggle/input/data-in-brief/comment_data.csv'  # Replace with your file path

# Set the chunk size (number of rows per part)
chunk_size = 20000

# Initialize a list to store the chunks as DataFrames
chunks_list = []

# Read the CSV file in chunks and store each chunk in the list
for chunk in pd.read_csv(csv_file, chunksize=chunk_size):
    # Append the chunk to the list
    chunks_list.append(chunk)
    
print(f'Total chunks created: {len(chunks_list)}')

# Iterate over the list of chunks and perform further operations
for i, chunk in enumerate(chunks_list):
    # Perform your operations on each chunk
    print(f'Processing chunk {i + 1} with {len(chunk)} rows')
    
    # Example: Save each chunk to a CSV file
    chunk_filename = f'part_{i + 1}.csv'
    chunk.to_csv(chunk_filename, index=False)
    
    print(f'Saved {chunk_filename}')


Total chunks created: 13
Processing chunk 1 with 20000 rows
Saved part_1.csv
Processing chunk 2 with 20000 rows
Saved part_2.csv
Processing chunk 3 with 20000 rows
Saved part_3.csv
Processing chunk 4 with 20000 rows
Saved part_4.csv
Processing chunk 5 with 20000 rows
Saved part_5.csv
Processing chunk 6 with 20000 rows
Saved part_6.csv
Processing chunk 7 with 20000 rows
Saved part_7.csv
Processing chunk 8 with 20000 rows
Saved part_8.csv
Processing chunk 9 with 20000 rows
Saved part_9.csv
Processing chunk 10 with 20000 rows
Saved part_10.csv
Processing chunk 11 with 20000 rows
Saved part_11.csv
Processing chunk 12 with 20000 rows
Saved part_12.csv
Processing chunk 13 with 17497 rows
Saved part_13.csv


In [4]:
pip install git+https://github.com/csebuetnlp/normalizer

  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-s54ujgak
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-s54ujgak
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl size=6859 sha256=7f477b1a0d7f0cdeb879bf4eb11094b5aa7f58f29b11165327cd26dcde8d4d89
  Stored in directory: /tmp/pip-ephem-wheel-cache-vmsqelqj/wheels/2e/79/9c/cd96d490298305d51d2da11484bb2c25fd1f759a6906708282
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186460 sha256=bd71d56f469cd79bacb364eadcd0d92239cb36b1

In [5]:
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from normalizer import normalize
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.notebook import tqdm
import torch
tqdm.pandas()

# Load model and tokenizer, ensure you are on GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Enable mixed precision for faster computation if supported by the GPU
model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/banglat5_nmt_bn_en").to(device).half()
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglat5_nmt_bn_en", use_fast=False)

def translate_text(text):
    # Normalize the text
    normalized_text = normalize(text)
    
    # Tokenize the normalized text
    inputs = tokenizer(normalized_text, return_tensors="pt", max_length=512, truncation=True).to(device)

    # Perform translation with GPU, no gradients, and half precision
    with torch.no_grad():
        generated_tokens = model.generate(
            inputs.input_ids,
            max_length=512,
            do_sample=False,  # Greedy decoding for faster results
            early_stopping=True
        )

    # Decode and return the translation
    translation = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
    
    return translation


config.json:   0%|          | 0.00/766 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [6]:
import re

def is_bengali(sentence):
    # Bengali Unicode range: U+0980 to U+09FF
    if re.search(r'[\u0980-\u09FF]', sentence):
        return True
    else:
        return False



    
def translate_bengali(text):
    """
    Translates the text to Bengali if the whole summary is written in English or 
    in mix of Bangla-English-Banglish.

    Args:
    - text (str): The input text to be processed.

    Returns:
    str: The translated text.
    """
    re.sub(r"'", '',text)
    split_text = text.split(' ')
   

    translated_text = []
    i = 0

    while i < len(split_text):
        if is_bengali(split_text[i]):
            temp = ""
            j=i
            while j<len(split_text):
                if is_bengali(split_text[j]):
                    temp += " " + split_text[j]
                    
                else:
                    i = j-1
                    break
                j+=1
                         # Translate the English words
            i=j-1
            translated_word = translate_text(temp.strip())
            translated_text.append(translated_word)
        else:
            translated_text.append(split_text[i])

        i += 1

    return ' '.join(translated_text)


In [11]:
tqdm.pandas()
for index,value in enumerate(chunks_list):
    value['translated']=value['text'].progress_apply(translate_bengali)
    save_filename = f'translated_{i + 1}.csv'
    chunk.to_csv(save_filename, index=False)

  0%|          | 0/20000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
data.to_csv('translated.csv',index=False)